In [42]:
#ashley brown, this variant of the program trains maths.csv against portuguese.csv

#drive.mount('/content/drive/')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
import imageio
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression 
from sklearn import datasets
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms

In [43]:
math = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/IntroToMachineLearning/Maths.csv")) 
portuguese = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/IntroToMachineLearning/Portuguese.csv")) 
math.head() 

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
m = len(math) 
m 

395

In [46]:
math.shape 

(395, 33)

In [47]:
math.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [48]:
math.describe().T

,count,mean,std,min,25%,50%,75%,max
age,395.0,16.696203,1.276043,15.0,16.0,17.0,18.0,22.0
Medu,395.0,2.749367,1.094735,0.0,2.0,3.0,4.0,4.0
Fedu,395.0,2.521519,1.088201,0.0,2.0,2.0,3.0,4.0
traveltime,395.0,1.448101,0.697505,1.0,1.0,1.0,2.0,4.0
studytime,395.0,2.035443,0.839240,1.0,1.0,2.0,2.0,4.0
failures,395.0,0.334177,0.743651,0.0,0.0,0.0,0.0,3.0
famrel,395.0,3.944304,0.896659,1.0,4.0,4.0,5.0,5.0
freetime,395.0,3.235443,0.998862,1.0,3.0,3.0,4.0,5.0
goout,395.0,3.108861,1.113278,1.0,2.0,3.0,4.0,5.0
Dalc,395.0,1.481013,0.890741,1.0,1.0,1.0,2.0,5.0


In [49]:
math.nunique()

school         2
sex            2
age            8
address        2
famsize        2
Pstatus        2
Medu           5
Fedu           5
Mjob           5
Fjob           5
reason         4
guardian       3
traveltime     4
studytime      4
failures       4
schoolsup      2
famsup         2
paid           2
activities     2
nursery        2
higher         2
internet       2
romantic       2
famrel         5
freetime       5
goout          5
Dalc           5
Walc           5
health         5
absences      34
G1            17
G2            17
G3            18
dtype: int64

In [50]:
math.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [51]:
math.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [52]:
num_cols=math.select_dtypes(include=['float64', 'int64'])
num_cols.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,18,4,4,2,2,0,4,3,4,1,1,3,6,5,6,6
1,17,1,1,1,2,0,5,3,3,1,1,3,4,5,5,6
2,15,1,1,1,2,3,4,3,2,2,3,3,10,7,8,10
3,15,4,2,1,3,0,3,2,2,1,1,5,2,15,14,15
4,16,3,3,1,2,0,4,3,2,1,2,5,4,6,10,10


In [53]:
cat_cols=math.select_dtypes(include=['object'])
cat_cols.head()

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,GP,F,U,GT3,A,at_home,teacher,course,mother,yes,no,no,no,yes,yes,no,no
1,GP,F,U,GT3,T,at_home,other,course,father,no,yes,no,no,no,yes,yes,no
2,GP,F,U,LE3,T,at_home,other,other,mother,yes,no,yes,no,yes,yes,yes,no
3,GP,F,U,GT3,T,health,services,home,mother,no,yes,yes,yes,yes,yes,yes,yes
4,GP,F,U,GT3,T,other,other,home,father,no,yes,yes,no,yes,yes,no,no


In [54]:
math1=math.copy()

In [55]:
#need to clean up dataset
def clean_data(df):
    temp = df
    x = df.columns.tolist()   #turns the column titles into a list
    types = df.dtypes           #returns the type of each column
    for i in range(len(x)):
        if types[i] == 'str' or types[i] == 'object':
            temp[x[i]] = df[x[i]].astype('category').cat.codes
        else: 
            temp[x[i]] = df[x[i]]
    return temp

In [56]:
math = clean_data(math)
df = math
df

portuguese = clean_data(portuguese)
df2 = portuguese

In [57]:
# You can see that your dataset has many columns with values as 'Yes' or 'No'. 
# But in order to fit a regression line, we would need numerical values and not string.
# List of variables to map 
 
varlist =  ['age', 'sex', 'studytime', 'famsup', 'paid', 'activities', 'higher', 'internet', 'Dalc', 'Walc', 'absences', 'G3'] 
 
# Defining the map function 
#def binary_map(x): 
#    return x.map({'yes': 1, "no": 0, "M": 0, "F": 1}) 
 
# Applying the function to the math list 
math = math[varlist]

portuguese = portuguese[varlist]
 
# Check the math dataframe now 
math.head() 

,age,sex,studytime,famsup,paid,activities,higher,internet,Dalc,Walc,absences,G3
0,18,0,2,0,0,0,1,0,1,1,6,6
1,17,0,2,1,0,0,1,1,1,1,4,6
2,15,0,2,0,1,0,1,1,2,3,10,10
3,15,0,3,1,1,1,1,1,1,1,2,15
4,16,0,2,1,1,0,1,0,1,2,4,10


In [58]:
#Splitting the Data into Training and Testing Sets 
from sklearn.model_selection import train_test_split 
 
# We specify this so that the train and test data set always have the same rows, respec
np.random.seed(0) 
#df_train, df_test = train_test_split(math, train_size = 0.8, test_size = 0.2, random_state=42)
df_train = math
df_train.shape

(395, 12)

In [59]:
#df_test.shape 

In [60]:
num_vars = ['age', 'sex', 'studytime', 'famsup', 'paid', 'activities', 'higher', 'internet', 'Dalc', 'Walc', 'absences', 'G3'] 
df_Newtrain = df_train[num_vars] 
#df_Newtest = df_test[num_vars] 
df_Newtrain.head() 

df_Newtest = df2[num_vars] #port

In [61]:
df_Newtrain.shape 

(395, 12)

In [62]:
# Here we can see that except for area, all the columns have small integer values.  
#So it is extremely important to rescale the variables so that they have a comparable s
#If we don't have comparable scales, then some of the coefficients as obtained by fitti
#This might become very annoying at the time of model evaluation.  
##So it is advised to use standardization or normalization so that the units of the coef
 
#As you know, there are two common ways of rescaling:
#1. Min-Max scaling 
#2. Standardisation (mean-0, sigma-1) 
 
import warnings 
warnings.filterwarnings('ignore') 
 
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
 
# define standard scaler 
#scaler = StandardScaler() 
scaler = MinMaxScaler() 
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])
df_Newtrain.head(20) 

df2[num_vars] = scaler.fit_transform(df2[num_vars])

In [63]:
y_Newtrain = df_Newtrain.pop('G3') 
x_Newtrain = df_Newtrain
y_Newtest = df_Newtest.pop('G3')
x_Newtest = df_Newtest

ydf2 = df2.pop('G3')
ydf2 = torch.tensor(ydf2.values).float()
ydf2 = ydf2[:,None]

In [64]:
y_Newtrain = torch.tensor(y_Newtrain.values).float()
y_Newtrain = y_Newtrain[:,None]
x_Newtrain = torch.tensor(x_Newtrain.values).float()
y_Newtest = torch.tensor(y_Newtest.values).float()
y_Newtest = y_Newtest[:,None]
x_Newtest = torch.tensor(x_Newtest.values).float()

In [65]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)

        #loss_val = loss_fn(t_p_val, port)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            

In [66]:
seq_model1 = nn.Sequential(
    nn.Linear(11,316),
    nn.Tanh(),
    nn.Linear(316, 11))
seq_model1

Sequential(
  (0): Linear(in_features=11, out_features=316, bias=True)
  (1): Tanh()
  (2): Linear(in_features=316, out_features=11, bias=True)
)

In [67]:
#seq_model2 = nn.Sequential(
#    nn.Linear(5,8),
 #   nn.Tanh(),
  #  nn.Linear(8,4),
   # nn.Tanh(),
    #nn.Linear(4,2),
    #nn.Tanh(),
    #nn.Linear(2, 5))
#seq_model2

In [68]:
optimizer = optim.SGD(seq_model1.parameters(), lr=1e-3)
training_loop(200, optimizer, seq_model1, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)

Epoch 1, Training loss 0.3814, Validation loss 0.4953
Epoch 10, Training loss 0.3504, Validation loss 0.4612
Epoch 20, Training loss 0.3195, Validation loss 0.4267
Epoch 30, Training loss 0.2920, Validation loss 0.3955
Epoch 40, Training loss 0.2674, Validation loss 0.3672
Epoch 50, Training loss 0.2455, Validation loss 0.3416
Epoch 60, Training loss 0.2259, Validation loss 0.3184
Epoch 70, Training loss 0.2085, Validation loss 0.2974
Epoch 80, Training loss 0.1929, Validation loss 0.2783
Epoch 90, Training loss 0.1790, Validation loss 0.2609
Epoch 100, Training loss 0.1666, Validation loss 0.2451
Epoch 110, Training loss 0.1555, Validation loss 0.2307
Epoch 120, Training loss 0.1456, Validation loss 0.2177
Epoch 130, Training loss 0.1367, Validation loss 0.2057
Epoch 140, Training loss 0.1288, Validation loss 0.1949
Epoch 150, Training loss 0.1218, Validation loss 0.1849
Epoch 160, Training loss 0.1155, Validation loss 0.1759
Epoch 170, Training loss 0.1099, Validation loss 0.1676
Epo

In [69]:
#optimizer = optim.SGD(seq_model2.parameters(), lr=1e-3)
#training_loop(200, optimizer, seq_model2, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)